# 第一部分: Sequential 的逐層包裝與應用 - 以 CNN 圖形辨識模型為例

手寫辨識模型將是今日作為範例的基本模型之一，我們一樣先從基本套件引用開始。

In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


## 1. 讀入套件、準備資料

和之前的範例一樣，我們先將 Keras 內所需要的函數讀進來。

In [4]:
# Keras 相關套件/函數
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adadelta


# 資料庫
from keras.datasets import mnist

## 2. 讀取 MNIST 資料庫

一如之前的課程，在一開始，我們先讀取 MNIST 手寫辨識的資料庫，並進行資料格式上的整理

In [ ]:
(x0_train, y0_train), (x0_test, y0_test) = mnist.load_data()

In [7]:
x0_train.shape

(60000, 28, 28)

做 CNN 的時候我們可以直接把矩陣塞進去。不過一張彩色的圖通常有 R, G, B 三個矩陣, 但我們這是灰階只有一個。所以 (28, 28) 的矩陣要變成

* channels_last: (28, 28, 1)
* channels_first: (1, 28, 28)

注意，這相當惱人，因為不同 backend (TensorFlow, Theano, CNTK) 的表示法是不一樣的！

In [8]:
K.image_data_format()

'channels_last'

In [9]:
if K.image_data_format() == 'channels_first':
    x_train = x0_train.reshape(60000, 1, 28, 28)
    x_test = x0_test.reshape(10000, 1, 28, 28)
else:
    x_train = x0_train.reshape(60000, 28, 28, 1)
    x_test = x0_test.reshape(10000, 28, 28, 1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train / 255
x_test = x_test / 255

In [10]:
y_train = np_utils.to_categorical(y0_train, 10)
y_test = np_utils.to_categorical(y0_test, 10)

## 2. 打造 CNN

我們使用和上次一樣的神經網路結構，但在每一個捲積層和全連接層，我們都使用指令 name= 來給定名字。

In [11]:
model = Sequential()

# convolution and then pooling
model.add(Conv2D(10, (3, 3), padding='same', input_shape=(28,28,1), name='first_conv_layer'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# convolution and then pooling
model.add(Conv2D(20, (3, 3), padding='same', name='second_conv_layer'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# convolution and then pooling
model.add(Conv2D(120, (3, 3), padding='same', name='third_conv_layer'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten and connect with a fully connected layer
model.add(Flatten())
model.add(Dense(200, name='fc_layer_200'))
model.add(Activation('relu'))

# conneted with smaller fully connected layer
# with the same number of neurons as the number of classes
model.add(Dense(10, name='fc_layer_10'))
model.add(Activation('softmax'))

In [12]:
model.compile(loss="categorical_crossentropy",
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv_layer (Conv2D)    (None, 28, 28, 10)        100       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 10)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 10)        0         
_________________________________________________________________
second_conv_layer (Conv2D)   (None, 14, 14, 20)        1820      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 20)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
third_conv_layer (Conv2D)    (None, 7, 7, 120)         21720     
__________

## 3. 重新打造 CNN

## 其實，若神經網路的結構具有區塊性，我們可透過分段定義的方式來建立局部建立，以下是透過區塊定義的方式，設計和上面手寫辨識模型一樣結構的神經網路。

### a) 首先，我們先將三層卷積層都用 list 包在一起。

In [15]:
conv_layer = [
    # convolution and then pooling
    Conv2D(10, (3, 3), padding='same', input_shape=(28,28,1), name='first_conv_layer'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # convolution and then pooling
    Conv2D(20, (3, 3), padding='same', name='second_conv_layer'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # convolution and then pooling
    Conv2D(120, (3, 3), padding='same', name='third_conv_layer'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2))
]

### 將剩下兩個全連接層用 list 包在一起

In [16]:
fc_layer = [
    # flatten and connect with a fully connected layer
    Flatten(),
    Dense(200, name='fc_layer_200'),
    Activation('relu'),

    # conneted with smaller fully connected layer
    # with the same number of neurons as the number of classes
    Dense(10, name='fc_layer_10'),
    Activation('softmax')
]

### b) 接著，用 Sequential 將兩個 list "加"起來

In [17]:
model_block_addition = Sequential(conv_layer + fc_layer)

In [18]:
model_block_addition.compile(loss="categorical_crossentropy",
              optimizer=Adadelta(),
              metrics=['accuracy'])

### 提示: 可以用 .summary() 來比較 model 和 model_block_addition 的結構及權重數量

In [19]:
model_block_addition.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv_layer (Conv2D)    (None, 28, 28, 10)        100       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 10)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 10)        0         
_________________________________________________________________
second_conv_layer (Conv2D)   (None, 14, 14, 20)        1820      
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 20)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
third_conv_layer (Conv2D)    (None, 7, 7, 120)         21720     
__________

## 4. 模型局部的重複使用

要注意到的是，新的手寫辨識模型有兩個區塊 conv_layer 和 fc_layer，若將 fc_layer 的神經元數量詳細寫出，我們將其表示為:
　　　　
<p><center>
conv_layer -> (1080 -> 200 -> 10)
</center></p>

這樣的結構，可能有人可能會好奇，將模型改成

<p><center>
conv_layer -> 1080 -> 10
</center></p>

會不會就能得到一樣的效果，而我們又不想重新定義新的 conv_layer，此時，該如何做測試？

In [20]:
fc_single_layer = [
    Flatten(),
    Dense(10, name='fc_layer_10'),
    Activation('softmax')
]

In [21]:
model_2 = Sequential(conv_layer + fc_single_layer)
model_2.compile(loss="categorical_crossentropy",
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [22]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv_layer (Conv2D)    (None, 28, 28, 10)        100       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 10)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 10)        0         
_________________________________________________________________
second_conv_layer (Conv2D)   (None, 14, 14, 20)        1820      
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 20)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
third_conv_layer (Conv2D)    (None, 7, 7, 120)         21720     
__________

### a) 首先，我們先看看兩個模型在"訓練前"對測試資料的正確率。

In [23]:
score = model_block_addition.evaluate(x_test, y_test)
score_2 = model_2.evaluate(x_test, y_test)

 9760/10000 [============================>.] - ETA: 0s

In [24]:
print("原始模型的正確率: ", score[1])
print("修改模型的正確率: ", score_2[1])

原始模型的正確率:  0.0977
修改模型的正確率:  0.028


### b) 再來，我們訓練原本的模型並查看其在測試資料上的正確率。

In [25]:
model_block_addition.fit(x_train, y_train, verbose=1, batch_size=128, epochs=1)

Epoch 1/1
60000/60000 [==============================] - 9s - loss: 0.3129 - acc: 0.9034     


In [26]:
score = model_block_addition.evaluate(x_test, y_test)
print("")
print("原始模型的準確率: ", score[1])

 9920/10000 [============================>.] - ETA: 0s
原始模型的準確率:  0.9769


### c) 接著，在不訓練修改模型的情況下，直接看修改模型在測試資料上的正確率。

In [27]:
score_2 = model_2.evaluate(x_test, y_test)
print(" ")
print("修改模型的準確率: ", score_2[1])

 9856/10000 [============================>.] - ETA: 0s 
修改模型的準確率:  0.0829


### d) 現在反過來，先訓練修改模型並查看其在測試資料上的正確率。

In [28]:
model_2.fit(x_train, y_train, verbose=1, batch_size=128, epochs=1)

Epoch 1/1
60000/60000 [==============================] - 8s - loss: 0.2594 - acc: 0.9306     


In [29]:
score_2 = model_2.evaluate(x_test, y_test)
print("修改模型的準確率: ", score_2[1])

 9792/10000 [============================>.] - ETA: 0s修改模型的準確率:  0.9729


### e) 重新查看原始模型在測試資料上的正確率。

In [30]:
score = model_block_addition.evaluate(x_test, y_test)
print("原始模型的準確率: ", score[1])

 9696/10000 [============================>.] - ETA: 0s原始模型的準確率:  0.9433


# <center>世界為何變了?</center>

In [31]:
for item in conv_layer:
    item.trainable = False
    
conv_layer[0].trainable
#可以變更

True

# 第二部分: `Model` functional API 的使用

首先，我們需要額外的兩個函數，分別是 `Model` 和 `Inout` 層。

In [28]:
from keras.models import Model
from keras.layers import Input

回憶之前模型的 summary ， output_shape 的改變是這樣的：

<table border="0.1">
　<tr>
　<td></td>
　<td>C</td>
  <td></td>
　<td>P</td>
  <td></td>
　<td>C</td>
  <td></td>
　<td>P</td> 
  <td></td>
　<td>C</td> 
  <td></td>
　<td>P</td> 
  <td></td>
　<td>F</td> 
   <td></td>
　<td>D</td> 
   <td></td>
  <td>D</td> 
  <td></td> 
　</tr>
　<tr>
　<td>28x28x1</td>
　<td>-></td>
  <td>28x28x10</td>
　<td>-></td>
  <td>14x14x10</td>
　<td>-></td>
  <td>14x14x20</td>
　<td>-></td> 
  <td>7x7x20</td>
　<td>-></td> 
  <td>7x7x120</td>
　<td>-></td> 
  <td>3x3x120</td>  
　<td>-></td> 
  <td>1080</td>
　<td>-></td> 
   <td>200</td>
　<td>-></td> 
   <td>10</td>
　</tr>
　<tr>
　<td>$x$</td>
　<td>-></td>
  <td>$u_1$</td>
　<td>-></td>
  <td>$u_2$</td>
　<td>-></td>
  <td>$v_1$</td>
　<td>-></td> 
  <td>$v_2$</td>
　<td>-></td> 
  <td>$s_1$</td>
　<td>-></td> 
  <td>$s_2$</td>  
　<td>-></td> 
  <td>$d_1$</td>
　<td>-></td> 
   <td>$d_2$</td>
　<td>-></td> 
   <td>y</td>
　</tr> 
</table>


Functional API 的意思就是讓所有的層指令，看成像是函數作用在某個變數上一樣，在這裡，每一個神經網路層都視為函數並作用在前一層變數上，為了辨別出不同的層的差異，除了 `MaxPooling2D` 之外，我們都用一個函數名稱來標記，則表格如下：

<table border="1">
　<tr>
　<td></td>
　<td>$f$</td>
  <td></td>
　<td>P</td>
  <td></td>
　<td>$g$</td>
  <td></td>
　<td>P</td> 
  <td></td>
　<td>$h$</td> 
  <td></td>
　<td>P</td> 
  <td></td>
　<td>Flat</td> 
   <td></td>
　<td>$\alpha$</td> 
   <td></td>
  <td>$\beta$</td> 
  <td></td> 
　</tr>
　<tr>
　<td>28x28x1</td>
　<td>-></td>
  <td>28x28x10</td>
　<td>-></td>
  <td>14x14x10</td>
　<td>-></td>
  <td>14x14x20</td>
　<td>-></td> 
  <td>7x7x20</td>
　<td>-></td> 
  <td>7x7x120</td>
　<td>-></td> 
  <td>3x3x120</td>  
　<td>-></td> 
  <td>1080</td>
　<td>-></td> 
   <td>200</td>
　<td>-></td> 
   <td>10</td>
　</tr>
　<tr>
　<td>$x$</td>
　<td>-></td>
  <td>$u_1$</td>
　<td>-></td>
  <td>$u_2$</td>
　<td>-></td>
  <td>$v_1$</td>
　<td>-></td> 
  <td>$v_2$</td>
　<td>-></td> 
  <td>$s_1$</td>
　<td>-></td> 
  <td>$s_2$</td>  
　<td>-></td> 
  <td>$d_1$</td>
　<td>-></td> 
   <td>$d_2$</td>
　<td>-></td> 
   <td>y</td>
　</tr> 
</table>

接下來，我們會使用 Model 建立和之前一樣的 CNN 手寫辨識模型，首先，是神經網路的輸入(Inout)，別忘記了， 輸入的資料大小為 28 x 28 x 1。

In [29]:
x = Input(shape=(28,28,1))

回憶一下手寫辨識模型的第一層是 

<p><center>Conv2D(10, (3, 3), padding='same', input_shape=(28,28,1), name='first_conv_layer')</center></p>

此時，第一層的 inpu_shape 將不被使用，此外，為了方便，我們將其 Activation 是 ReLu，我們將其合併在一起，寫成

<p><center>Conv2D(10, (3, 3), padding='same', activation='relu', name='first_conv_layer')</center></p>


Functional API 的意思就是讓所有的層指令，看成像是函數作用在某個變數上一樣，在這裡，`Conv2D(...)` 會作為一個函數，作用在變數 $x$ 上。為了方便起見，給 `Conv2D(...)` 一個好看的名字 $f$，並作用在 $x$ 上得到 $f(x)$。

In [30]:
f = Conv2D(10, (3, 3), padding='same', activation='relu', name='first_conv_layer')
u_1 = f(x)

我們剛剛處理的，是表格中的:

<table border="1">
　<tr>
　<td></td>
　<td bgcolor="#FFFF00">$f$</td>
  <td></td>
　<td>P</td>
  <td></td>
　<td>$g$</td>
  <td></td>
　<td>P</td> 
  <td></td>
　<td>$h$</td> 
  <td></td>
　<td>P</td> 
  <td></td>
　<td>Flat</td> 
   <td></td>
　<td>$\alpha$</td> 
   <td></td>
  <td>$\beta$</td> 
  <td></td> 
　</tr>
　<tr>
　<td>28x28x1</td>
　<td>-></td>
  <td>28x28x10</td>
　<td>-></td>
  <td>14x14x10</td>
　<td>-></td>
  <td>14x14x20</td>
　<td>-></td> 
  <td>7x7x20</td>
　<td>-></td> 
  <td>7x7x120</td>
　<td>-></td> 
  <td>3x3x120</td>  
　<td>-></td> 
  <td>1080</td>
　<td>-></td> 
   <td>200</td>
　<td>-></td> 
   <td>10</td>
　</tr>
　<tr>
　<td bgcolor="#FFFF00">$x$</td>
　<td bgcolor="#FFFF00">-></td>
  <td bgcolor="#FFFF00">$u_1$</td>
　<td>-></td>
  <td>$u_2$</td>
　<td>-></td>
  <td>$v_1$</td>
　<td>-></td> 
  <td>$v_2$</td>
　<td>-></td> 
  <td>$s_1$</td>
　<td>-></td> 
  <td>$s_2$</td>  
　<td>-></td> 
  <td>$d_1$</td>
　<td>-></td> 
   <td>$d_2$</td>
　<td>-></td> 
   <td>y</td>
　</tr> 
</table>

接著是 `MaxPooling` 的部分。

In [31]:
P = MaxPooling2D(pool_size=(2, 2))
u_2 = P(u_1)

注意到，因為模型中所有的 `MaxPooling2D(pool_size=(2, 2))` 都長的一樣且不帶有參數，因此，定義過一次之後，就能重複使用。

同上，我們已經處理完的部分是:

<table border="1">
　<tr>
　<td></td>
　<td>f</td>
  <td></td>
　<td bgcolor="#FFFF00">P</td>
  <td></td>
　<td>g</td>
  <td></td>
　<td>P</td> 
  <td></td>
　<td>h</td> 
  <td></td>
　<td>P</td> 
  <td></td>
　<td>Flat</td> 
   <td></td>
　<td>$\alpha$</td> 
   <td></td>
  <td>$\beta$</td> 
  <td></td> 
　</tr>
　<tr>
　<td>28x28x1</td>
　<td>-></td>
  <td>28x28x10</td>
　<td>-></td>
  <td>14x14x10</td>
　<td>-></td>
  <td>14x14x20</td>
　<td>-></td> 
  <td>7x7x20</td>
　<td>-></td> 
  <td>7x7x120</td>
　<td>-></td> 
  <td>3x3x120</td>  
　<td>-></td> 
  <td>1080</td>
　<td>-></td> 
   <td>200</td>
　<td>-></td> 
   <td>10</td>
　</tr>
　<tr>
　<td>$x$</td>
　<td>-></td>
  <td bgcolor="#FFFF00">$u_1$</td>
　<td bgcolor="#FFFF00">-></td>
  <td bgcolor="#FFFF00">$u_2$</td>
　<td>-></td>
  <td>$v_1$</td>
　<td>-></td> 
  <td>$v_2$</td>
　<td>-></td> 
  <td>$s_1$</td>
　<td>-></td> 
  <td>$s_2$</td>  
　<td>-></td> 
  <td>$d_1$</td>
　<td>-></td> 
   <td>$d_2$</td>
　<td>-></td> 
   <td>y</td>
　</tr> 
</table>
剩下的部分依同樣的方式處理，則程式碼如下:

In [32]:
#convolution+polling
g = Conv2D(20, (3, 3), padding='same', activation='relu', name='second_conv_layer')
v_1 = g(u_2)
v_2 = P(v_1)


h = Conv2D(120, (3, 3), padding='same', activation='relu', name='third_conv_layer')
s_1 = h(v_2)
s_2 = P(s_1)

Flat = Flatten()
d_1 = Flat(s_2)
    
alpha = Dense(200, activation='relu', name='fc_layer_200')
d_2 = alpha(d_1)

beta = Dense(10, activation='relu', name='fc_layer_10')
y = beta(d_2)

接著，在 `Model` 指令中，將輸入 `x` 和輸出 `y` 指定，就可以構成一個模型了！

In [33]:
model_functional = Model(x, y)

In [34]:
model_functional.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
first_conv_layer (Conv2D)    (None, 28, 28, 10)        100       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
second_conv_layer (Conv2D)   (None, 14, 14, 20)        1820      
_________________________________________________________________
third_conv_layer (Conv2D)    (None, 7, 7, 120)         21720     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1080)              0         
_________________________________________________________________
fc_layer_200 (Dense)         (None, 200)               216200    
__________

# 第三部分: Autoencoder 的介紹與 Submodel 的應用

首先，我們需要引進 `Reshape` layer，這個功能和 Numpy 的 reshape 很類似，只是是做為神經網路層來作用的。

In [35]:
from keras.layers.core import Reshape

# 繪圖所需的運算函數
from scipy.stats import norm

In [36]:
x_train_reshape = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test_reshape = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

### a) Autoencoder的建置與訓練: 在這個部分，我們將使用 `Model` 的寫法來撰寫 Autoencoder，並將 encoder 和 decoder 兩個子模型分別從原始模型中定義出來。

In [37]:
#autoencoder:784->100->2->100->784
x = Input(shape=(784, ))

f = Dense(100, activation='sigmoid', name='hidden_layer_encoder')
h_1 = f(x)

encoder = Dense(2, activation='relu', name='representation_layer')
encoded_x = encoder(h_1)

g = Dense(100, activation='sigmoid', name='hidden_layer_decoder')
h_2 = g(encoded_x)

h = Dense(784, activation='sigmoid')
x_reconstructed = h(h_2)

#reshape_from_784_to_28x28x1 = Reshape((28, 28, 1))
#x_reconstructed = reshape_from_784_to_28x28x1(deconded_h)

autoencoder = Model(x, x_reconstructed)

In [38]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 784)               0         
_________________________________________________________________
hidden_layer_encoder (Dense) (None, 100)               78500     
_________________________________________________________________
representation_layer (Dense) (None, 2)                 202       
_________________________________________________________________
hidden_layer_decoder (Dense) (None, 100)               300       
_________________________________________________________________
dense_1 (Dense)              (None, 784)               79184     
Total params: 158,186.0
Trainable params: 158,186.0
Non-trainable params: 0.0
_________________________________________________________________


從 summary 可以看出除了一開始的 `Flatten` 和最後的 `Reshape`，Autoencoder 最主要的目的就是逐漸將原始資料壓縮，然後再反向的放大回來，並且希望最後得到的，和原本的檔案一樣。

若能重建檔案，就代表中間的(壓縮 -> 還原)的過程，能保留檔案內(抽象)的重建資訊，因此，只需要紀錄檔案在神經網路中間的數值即可。

In [ ]:
#unsupervised learning
autoencoder.compile(loss="binary_crossentropy", optimizer='sgd')

In [ ]:
autoencoder.fit(x_train_reshape, x_train_reshape, verbose=1, batch_size=128, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.6897     
Epoch 2/10
60000/60000 [==============================] - 3s - loss: 0.6653     
Epoch 3/10
60000/60000 [==============================] - 3s - loss: 0.6427     
Epoch 4/10
11008/60000 [====>.........................] - ETA: 3s - loss: 0.6301

### b) Encoder 子模型的定義及視覺化

In [ ]:
x_train_reconstructed = autoencoder.predict(x_train_reshape)

In [ ]:
x_train_reconstructed.shape

In [ ]:
x_train_reconstructed = x_train_reconstructed.reshape(60000, 28, 28)

In [ ]:
no_test = np.random.randint(0,60000)
plt.subplot(1,2,1)
plt.imshow(x_train[no_test, :, :, 0], cmap='Greys')
plt.subplot(1,2,2)
plt.imshow(x_train_reconstructed[no_test], cmap='Greys')

經過訓練後，我們可以看到圖片以及重建後的樣子，但我們關注的，並不是網路的輸出，而是其中間值，因此，我們定義一個小模型，使其輸出為原本 Autoencoder 的中間層，我們稱之為潛在特徵表示法 (latent feature representation)。

In [ ]:
input_img = autoencoder.input
reconstructed_img = autoencoder.layers[3]

In [ ]:
encoder = Model(x, encoded_x)

In [ ]:
x_train_encoded = encoder.predict(x_train_reshape)

In [ ]:
x_train_encoded.shape

In [ ]:
# display a 2D plot of the digit classes in the latent space
plt.figure(figsize=(6, 6))
plt.scatter(x_train_encoded[:, 0], x_train_encoded[:, 1], c=9-y0_train)
plt.colorbar()
plt.show()

In [ ]:
x_train_encoded_small = encoder.predict(x_train_reshape[:1000])
# display a 2D plot of the digit classes in the latent space
plt.figure(figsize=(6, 6))
plt.scatter(x_train_encoded_small[:, 0], x_train_encoded_small[:, 1], c=9-y0_train[:1000])
plt.colorbar()
plt.show()

可以看出，從某些不同數字的潛在特徵表示法，並沒有在空間上的關聯性，例如: 8 和 9 的潛在特徵表示法應該要很接近，而且介於兩者之間的潛在特徵表示，應該要與 8 和 9 相似，但從圖中發現，並沒有這樣的現象，也因此，我們需要在潛在特徵表示法上具有一些連續性的限制。

為了檢驗上述的說法，我們視覺化 Autoencoder 的後半部，也就是 decoder 的部分，並將潛在特徵空間上進行均勻取樣並將結果貼在上面，用以檢視重建圖片的非連續性。

### c) Decoder 子模型的定義及視覺化

In [ ]:
# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(2,))
_h_decoded = g(decoder_input)
_x_decoded = h(_h_decoded)

generator = Model(decoder_input, _x_decoded)

如上面的程式碼，除了重新定義一個 2維的 Input 之外，剩下的函數都沿用 Autoencoder 已經定義過的層，這也是 Functional API 的精神所在，將任何運算都定義成函數的樣子去運作。

In [ ]:
# display a 2D surface of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

In [ ]:
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = generator.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()

### 神經網路沒有達到理想中的效果，該怎麼處理？
* 增加/減少隱藏層的神經元數量
* 增加隱藏層的數量，換句話說，就是改用 stacked Autoencoder
* 改變訓練方式 e.g., SGD, Adagrad, rmsprop...
* 改變權重的初始值
* 加入 Dropout 結構
* 層上面加入權重的限制式

# 第四部分: Variational Autoencoder (VAE) 的介紹

在這個部分，我們需要引進 `Lambda` layer 來客製化神經網路層，這是為了在 VAE 建立抽樣層。

In [ ]:
from keras.layers import Lambda

一開始，我們先把在這抽樣層之前的神經網路結構定義好，別忘記，在使用 `Model` 將神經網路的輸入輸出定義之前，這些變數都是自由的。

In [ ]:
x = Input(shape=(784, ))
h = Dense(100, activation='relu', name='hidden_layer_encoder')(x)

f = Dense(2, name='mean')
z_mean = f(h)

g = Dense(2, name='log_variance')
z_log_var = g(h)

在上面 z_mean 和 z_log_var 代表的是多元常態分配的 Mean 和取對數後的 Variance (為何要取對數?)，將這兩者做為抽樣層的輸入，並希望抽樣出以上述資訊作為常態分配的抽樣母體；接著，我們來定義抽樣層。

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(2, ), mean=0., stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

在使用 `Lambda` 指令定義客製化層的時候，必須告訴對方吐出來的 tensor 大小。

In [ ]:
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(2, ), name='Normal_Sampleing')   ([z_mean, z_log_var])

In [ ]:
# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(100, activation='relu', name='hidden_layer_decoder')
h_decoded = decoder_h(z)

decoder_mean = Dense(784, activation='sigmoid', name='decoded_vector')
x_decoded_mean = decoder_mean(h_decoded)

#reshape_from_784_to_28x28x1 = Reshape((28, 28, 1), name='decoded_image')
#x_decoded = reshape_from_784_to_28x28x1(x_decoded_mean)

vae = Model(x, x_decoded_mean)

In [ ]:
vae.summary()

注意到，VAE 的損失函數是檔案重建前後的 crossentropy，再加上中間的 (mean, var) 和 n維獨立標準常態分配的 KL-散度，因此，我們必須將中間的損失定出來，詳細定義如下：

In [ ]:
def vae_loss(x, x_decoded):
    reconstruct_loss = 784 * K.binary_crossentropy(x, x_decoded)
    
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
                            , axis=-1)
        
    return reconstruct_loss + kl_loss

In [ ]:
vae.compile(loss=vae_loss, optimizer=Adadelta())

為了節省時間，我使用已經訓練好的模型權重。

In [ ]:
vae.load_weights('vae_MNIST_weights.h5')
#vae.fit(x_train, x_train, shuffle=True, verbose=1, batch_size=128, epochs=1)

同樣的，我們來看 VAE 的 endocer 部分:

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

In [ ]:
# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test_reshape)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y0_test)
plt.colorbar()
plt.show()

同樣的，我們也觀察看看 decoder 的

In [ ]:
# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(2,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [ ]:
# display a 2D surface of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

In [ ]:
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = generator.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()